<a href="https://colab.research.google.com/github/santhosh12-a/511323106046-AI-Movie/blob/main/511323106046_delevering_peronalized_movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# Sample user-movie ratings data
ratings_dict = {
    'user_id': [1, 1, 1, 2, 2, 3, 3, 4],
    'movie': ['Inception', 'Interstellar', 'The Matrix', 'Inception', 'Tenet', 'The Matrix', 'Tenet', 'Interstellar'],
    'rating': [5, 4, 5, 5, 3, 4, 4, 2]
}

# Create DataFrame
df = pd.DataFrame(ratings_dict)

# Create user-item matrix
user_movie_matrix = df.pivot_table(index='user_id', columns='movie', values='rating').fillna(0)

# Normalize ratings
scaler = StandardScaler()
user_movie_matrix_scaled = scaler.fit_transform(user_movie_matrix)

# Compute similarity between users
user_similarity = cosine_similarity(user_movie_matrix_scaled)

# Convert similarity array to DataFrame
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

def get_similar_users(user_id, top_n=2):
    """Return top N similar users to the given user_id"""
    if user_id not in user_similarity_df.index:
        return []
    return user_similarity_df[user_id].sort_values(ascending=False).drop(user_id).head(top_n).index.tolist()

def recommend_movies(user_id, top_n=3):
    """Recommend movies for a user based on similar users' ratings"""
    similar_users = get_similar_users(user_id)
    if not similar_users:
        return []

    # Aggregate ratings from similar users
    similar_ratings = user_movie_matrix.loc[similar_users].mean()
    # Exclude movies already rated by the user
    rated_movies = user_movie_matrix.loc[user_id]
    unrated_movies = rated_movies[rated_movies == 0]
    recommendations = similar_ratings[unrated_movies.index].sort_values(ascending=False).head(top_n)

    return recommendations.index.tolist()

# Example usage
user_id = 1
print(f"Recommended movies for user {user_id}: {recommend_movies(user_id)}")
